In [1]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
def create_dataset(dataset,timestep):
    X=[]
    Y=[]
    for i in range(dataset.shape[0]-timestep):
        X.append(dataset.iloc[i:i+timestep,0])
        Y.append(dataset.iloc[i+timestep,0])
    return np.array(X),np.array(Y)

In [3]:
df=pd.read_csv('FUTSTK_TATAMOTORS_23-May-2024_TO_23-Aug-2024.csv')
df.head()

,Symbol,Date,Expiry,Open,High,Low,Close,LTP,Settle Price,No. of contracts,Turnover * in ₹ Lakhs,Open Int,Change in OI,Underlying Value
0,TATAMOTORS,23-May-2024,30-May-2024,952.00,966.0,947.85,964.85,964.75,964.85,16158.0,220462.92,57750975.0,-4763775.0,962.35
1,TATAMOTORS,23-May-2024,27-Jun-2024,952.75,966.0,948.30,965.00,964.60,965.00,6731.0,91892.11,24306225.0,481650.0,962.35
2,TATAMOTORS,23-May-2024,25-Jul-2024,957.00,971.5,956.00,970.90,970.90,970.90,718.0,3811.17,1599400.0,-68750.0,962.35
3,TATAMOTORS,24-May-2024,30-May-2024,964.00,973.8,960.40,963.00,961.55,963.00,13394.0,184839.86,54802650.0,-2948325.0,960.55
4,TATAMOTORS,24-May-2024,27-Jun-2024,964.60,973.8,950.05,963.55,961.75,963.55,7577.0,104611.41,27419850.0,3113625.0,960.55


In [4]:
df1=df[['Date  ','Open  ']]
df1.rename(columns={'Date  ':'Date','Open  ':'Open'},inplace=True)
df1.columns,df1.head()

C:\Users\Kushagra\AppData\Local\Temp\ipykernel_8320\895408704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={'Date  ':'Date','Open  ':'Open'},inplace=True)


(Index(['Date', 'Open'], dtype='object'),
           Date    Open
 0  23-May-2024  952.00
 1  23-May-2024  952.75
 2  23-May-2024  957.00
 3  24-May-2024  964.00
 4  24-May-2024  964.60)

In [5]:
df1.dtypes

Date     object
Open    float64
dtype: object

In [6]:
import datetime


In [7]:
a=[]
for i in df1.Date:
    a.append(datetime.datetime.strptime(i,'%d-%b-%Y'))
df1.Date=a
df1.head()

C:\Users\Kushagra\AppData\Local\Temp\ipykernel_8320\3049488806.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.Date=a


,Date,Open
0,2024-05-23,952.00
1,2024-05-23,952.75
2,2024-05-23,957.00
3,2024-05-24,964.00
4,2024-05-24,964.60


In [8]:
df1.Date.describe()

count                              189
mean     2024-07-07 11:25:42.857142784
min                2024-05-23 00:00:00
25%                2024-06-13 00:00:00
50%                2024-07-08 00:00:00
75%                2024-07-31 00:00:00
max                2024-08-22 00:00:00
Name: Date, dtype: object

In [9]:
df1.set_index('Date',inplace=True)

In [10]:
train=df1[df1.index<'20240801']
test=df1[df1.index>'20240731']
train.shape,test.shape

((144, 1), (45, 1))

In [16]:
from sklearn.preprocessing import MinMaxScaler

In [21]:
sc=MinMaxScaler()
train1=pd.DataFrame(sc.fit_transform(train),index=train.index)

In [22]:
train1

,0
Date,
2024-05-23,0.131024
2024-05-23,0.133927
2024-05-23,0.150377
2024-05-24,0.177472
2024-05-24,0.179795
...,...
2024-07-30,0.829688
2024-07-30,0.842075
2024-07-31,0.952971


In [23]:
X,Y=create_dataset(train1,20)
X.shape,Y.shape

((124, 20), (124,))

In [24]:
from sklearn.model_selection import train_test_split
X_train,X_val,Y_train,Y_val=train_test_split(X,Y,test_size=20)

In [25]:
X_train=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_val=X_val.reshape(X_val.shape[0],X_val.shape[1],1)

In [26]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(20,1)))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

C:\Users\Kushagra\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 20, 50)              │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 20, 50)              │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 50)                  │          20,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              51 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 50,851 (198.64 KB)

 Trainable params: 50,851 (198.64 KB)

 Non-trainable params: 0 (0.00 B)

In [27]:
model.fit(X_train,Y_train,validation_data=(X_val,Y_val),epochs=100,verbose=1)

Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 288ms/step - loss: 0.1446 - val_loss: 0.0511
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0567 - val_loss: 0.0289
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0300 - val_loss: 0.0280
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0229 - val_loss: 0.0165
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0246 - val_loss: 0.0157
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.0205 - val_loss: 0.0162
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0188 - val_loss: 0.0223
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - loss: 0.0213 - val_loss: 0.0148
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 0.0182 - val_loss: 0.0119
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0195 - val_loss: 0.0105
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0174 - val_loss: 0.0116
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 0.0151 - val_l

In [29]:
Y_pred=model.predict(X)
Y_pred

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


array([[0.10559942],
       [0.10463926],
       [0.10752775],
       [0.12078914],
       [0.13920288],
       [0.15339206],
       [0.16079009],
       [0.16177781],
       [0.14394426],
       [0.11447306],
       [0.08567195],
       [0.06571166],
       [0.05785718],
       [0.06331007],
       [0.07323955],
       [0.0881616 ],
       [0.10620484],
       [0.13131827],
       [0.16585986],
       [0.20782407],
       [0.24155492],
       [0.26285973],
       [0.2707084 ],
       [0.2735968 ],
       [0.28080916],
       [0.29644203],
       [0.3093199 ],
       [0.31906044],
       [0.33142325],
       [0.32175142],
       [0.30272898],
       [0.28843495],
       [0.27813643],
       [0.27628252],
       [0.2880826 ],
       [0.2940971 ],
       [0.29751644],
       [0.303682  ],
       [0.29375607],
       [0.27711824],
       [0.2671508 ],
       [0.25690302],
       [0.25357342],
       [0.25838292],
       [0.24558848],
       [0.2248017 ],
       [0.20983608],
       [0.195

In [30]:
Y

array([0.10392878, 0.10392878, 0.17611767, 0.15598994, 0.1153474 ,
       0.12579834, 0.11999226, 0.        , 0.01799884, 0.03173989,
       0.06580221, 0.0803174 , 0.09773563, 0.06270563, 0.11689568,
       0.11573447, 0.19295529, 0.24811302, 0.2587575 , 0.21985678,
       0.23824269, 0.23843623, 0.29359396, 0.32784982, 0.36055738,
       0.32649506, 0.34855816, 0.38184633, 0.27056319, 0.31333462,
       0.34275208, 0.31604413, 0.3342365 , 0.36326689, 0.29591639,
       0.31662473, 0.34197794, 0.24772595, 0.26707954, 0.29475518,
       0.2485001 , 0.27578866, 0.2862396 , 0.15966712, 0.18792336,
       0.21521192, 0.1681827 , 0.18772982, 0.2161796 , 0.14766789,
       0.17379524, 0.19818076, 0.13857171, 0.16353784, 0.17747242,
       0.22779176, 0.26437004, 0.31875363, 0.29727114, 0.31739888,
       0.35436423, 0.32843042, 0.336946  , 0.36849236, 0.27811109,
       0.29727114, 0.32088252, 0.25469325, 0.27694987, 0.31043159,
       0.3439133 , 0.37062125, 0.3514612 , 0.3514612 , 0.35552

In [32]:
Y_pred[:,0]/Y

C:\Users\Kushagra\AppData\Local\Temp\ipykernel_8320\1249665555.py:1: RuntimeWarning: divide by zero encountered in divide
  Y_pred[:,0]/Y


array([1.01607485, 1.00683625, 0.61054491, 0.77433932, 1.20681421,
       1.2193489 , 1.34000383,        inf, 7.99741945, 3.60659939,
       1.30196161, 0.81814978, 0.59197635, 1.00963932, 0.62653767,
       0.76175753, 0.55041163, 0.52926795, 0.64098572, 0.9452702 ,
       1.01390272, 1.10243201, 0.92205034, 0.83451867, 0.77881962,
       0.90795257, 0.88742698, 0.83557289, 1.22493845, 1.02686199,
       0.88323018, 0.9126414 , 0.83215457, 0.76054969, 0.97352701,
       0.92885067, 0.86998722, 1.22587882, 1.09988232, 0.9401641 ,
       1.07505306, 0.93152135, 0.88587819, 1.61826004, 1.30685446,
       1.04455972, 1.24766747, 1.04267768, 0.86668961, 1.29801209,
       1.10726257, 0.9706618 , 1.41801768, 1.18857422, 1.06881556,
       0.8250664 , 0.73648869, 0.66796781, 0.83145337, 0.89065335,
       0.88362771, 1.03556907, 1.04834077, 0.95920874, 1.26422551,
       1.11087897, 0.93688698, 1.09007102, 0.91256507, 0.7552667 ,
       0.67758433, 0.67778593, 0.81491149, 0.92387343, 0.99656

In [69]:
df2=pd.DataFrame(sc.transform(df1))

In [70]:
X_test,Y_test=create_dataset(df2,20)
Y_pred1=model.predict(X_test)
Y_pred1[:,0]/Y_test

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 


C:\Users\Kushagra\AppData\Local\Temp\ipykernel_8320\1889301026.py:3: RuntimeWarning: divide by zero encountered in divide
  Y_pred1[:,0]/Y_test


array([1.01607485, 1.00683625, 0.61054491, 0.77433932, 1.20681421,
       1.2193489 , 1.34000383,        inf, 7.99741945, 3.60659939,
       1.30196161, 0.81814978, 0.59197635, 1.00963932, 0.62653767,
       0.76175753, 0.55041163, 0.52926795, 0.64098572, 0.9452702 ,
       1.01390272, 1.10243201, 0.92205034, 0.83451867, 0.77881962,
       0.90795257, 0.88742698, 0.83557289, 1.22493845, 1.02686199,
       0.88323018, 0.9126414 , 0.83215457, 0.76054969, 0.97352701,
       0.92885067, 0.86998722, 1.22587882, 1.09988232, 0.9401641 ,
       1.07505306, 0.93152135, 0.88587819, 1.61826004, 1.30685446,
       1.04455972, 1.24766747, 1.04267768, 0.86668961, 1.29801209,
       1.10726257, 0.9706618 , 1.41801768, 1.18857422, 1.06881556,
       0.8250664 , 0.73648869, 0.66796781, 0.83145337, 0.89065335,
       0.88362771, 1.03556907, 1.04834077, 0.95920874, 1.26422551,
       1.11087897, 0.93688698, 1.09007102, 0.91256507, 0.7552667 ,
       0.67758433, 0.67778593, 0.81491149, 0.92387343, 0.99656

In [72]:
sc.inverse_transform(Y_pred1)

array([[ 945.43164],
       [ 945.1836 ],
       [ 945.9298 ],
       [ 949.35583],
       [ 954.11304],
       [ 957.7789 ],
       [ 959.69006],
       [ 959.9453 ],
       [ 955.338  ],
       [ 947.7241 ],
       [ 940.2833 ],
       [ 935.12665],
       [ 933.0974 ],
       [ 934.50616],
       [ 937.0715 ],
       [ 940.9265 ],
       [ 945.588  ],
       [ 952.0761 ],
       [ 960.99994],
       [ 971.8413 ],
       [ 980.5557 ],
       [ 986.0598 ],
       [ 988.0875 ],
       [ 988.83374],
       [ 990.697  ],
       [ 994.7358 ],
       [ 998.0628 ],
       [1000.5792 ],
       [1003.7732 ],
       [1001.2745 ],
       [ 996.36   ],
       [ 992.6672 ],
       [ 990.0066 ],
       [ 989.5276 ],
       [ 992.5762 ],
       [ 994.13   ],
       [ 995.01337],
       [ 996.60626],
       [ 994.0419 ],
       [ 989.74347],
       [ 987.16846],
       [ 984.5209 ],
       [ 983.6607 ],
       [ 984.9032 ],
       [ 981.5978 ],
       [ 976.22754],
       [ 972.36115],
       [ 968.